In [1]:
## Pointwise Monotonic Loss function

In [2]:
import sys
sys.path.append("../src")
from Utils import readConfigurations

import json
import pandas
import importlib
import tensorflow as tf
from joblib import Parallel, delayed
import numpy as np

In [3]:
## Setting up the configurations ##

configuration_file = "../configurations/auto-mpg.txt"
configurations = readConfigurations(configuration_file)

## Some hacks for notebook
configurations["model_dir"] = "../examples/Auto-MPG/"
# print(json.dumps(configurations, indent = 4))

In [4]:
## Getting the data ##

from ModelCalls import generate_data, make_batch, evaluate_model, update_model
sys.path.append('../src/Models')
make_data = importlib.__import__(configurations['model']).make_data

train_all, train_labels_all, test_all, test_labels_all, min_max_dict = generate_data(make_data, configurations)

train_data = train_all[0]
train_labels = train_labels_all[0]
test_data = test_all[0]
test_labels = test_labels_all[0]

# train_labels['id'] = range(0, len(train_labels))
# print(train_labels)
# for x in train_labels:
#     x = 1
train_labels = np.asarray([([a,b]) for a,b in enumerate(train_labels)])
test_labels = np.asarray([([a,b]) for a,b in enumerate(test_labels)])

print(train_labels)
print(type(train_labels))

[[  0.   18. ]
 [  1.   15. ]
 [  2.   16. ]
 [  3.   17. ]
 [  4.   15. ]
 [  5.   14. ]
 [  6.   14. ]
 [  7.   15. ]
 [  8.   15. ]
 [  9.   14. ]
 [ 10.   14. ]
 [ 11.   24. ]
 [ 12.   22. ]
 [ 13.   26. ]
 [ 14.   25. ]
 [ 15.   24. ]
 [ 16.   25. ]
 [ 17.   26. ]
 [ 18.   10. ]
 [ 19.   10. ]
 [ 20.   11. ]
 [ 21.    9. ]
 [ 22.   27. ]
 [ 23.   28. ]
 [ 24.   25. ]
 [ 25.   19. ]
 [ 26.   16. ]
 [ 27.   17. ]
 [ 28.   19. ]
 [ 29.   18. ]
 [ 30.   14. ]
 [ 31.   14. ]
 [ 32.   14. ]
 [ 33.   14. ]
 [ 34.   12. ]
 [ 35.   13. ]
 [ 36.   18. ]
 [ 37.   23. ]
 [ 38.   28. ]
 [ 39.   30. ]
 [ 40.   30. ]
 [ 41.   31. ]
 [ 42.   35. ]
 [ 43.   27. ]
 [ 44.   26. ]
 [ 45.   24. ]
 [ 46.   25. ]
 [ 47.   20. ]
 [ 48.   21. ]
 [ 49.   13. ]
 [ 50.   17. ]
 [ 51.   11. ]
 [ 52.   13. ]
 [ 53.   12. ]
 [ 54.   13. ]
 [ 55.   19. ]
 [ 56.   15. ]
 [ 57.   13. ]
 [ 58.   13. ]
 [ 59.   22. ]
 [ 60.   21. ]
 [ 61.   26. ]
 [ 62.   22. ]
 [ 63.   28. ]
 [ 64.   23. ]
 [ 65.   28. ]
 [ 66.   1

In [16]:
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(7,)))
model.add(tf.keras.layers.Dense(12, activation='relu'))
model.add(tf.keras.layers.Dense(12, activation='relu'))
model.add(tf.keras.layers.Dense(1))

def custom_loss_envelope(model, x):
    x_tensor = tf.convert_to_tensor(x, dtype=tf.float32)      
        
    def custom_loss(y_true, y_pred):        
        
        correct_y_true = tf.gather(y_true, [1], axis=1)
        loss_PDE = tf.keras.losses.mean_squared_error(correct_y_true, y_pred)
#         return loss_PDE
#         monotonic_columns = tf.gather(x_tensor, [1], axis=1)
    
#         tf.print(type(x_tensor))
#         tf.print(type(monotonic_columns))
#         monotonic_columns = x_tensor
        
        with tf.GradientTape() as t:
            t.watch(x_tensor)
            output = model(x_tensor)
        
#         tf.print(output)
        x_indices = tf.gather(y_true, [0], axis=1)
        x_indices = tf.reshape(x_indices, [-1])
#         tf.print(x_indices)
        
#         monotonic_columns = tf.gather(x_tensor, [1], axis=1)
        
#         tf.print(x_tensor)
#         tf.print(monotonic_columns)
        
        DyDx = t.gradient(output, x_tensor)

        
        tf.print(DyDx)
# #         tf.print(DyDx[0])
# #         tf.print(DyDx[0].shape)
#         tf.print(x_indices)
        
# #         temp = DyDx[x_indices]
#         temp = tf.map_fn(lambda x: DyDx[int(x)], x_indices)
# # #         tf.print(temp)
        
#         main_row = tf.gather(temp, [1], axis=1)
#         main_row = tf.reshape(main_row, [-1])
# # #         tf.print(main_row)
# # #         tf.print("----JMD---")
#         final = tf.map_fn(lambda x: 0.0 if x < 0.0 else x, main_row)
    
#         tf.print(loss_PDE)
#         tf.print(final)
#         tf.print("------")
    
        return loss_PDE
    return custom_loss

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss=custom_loss_envelope(model, train_data))
model.fit(train_data, train_labels, epochs=25, batch_size=32, validation_split = 0.2)


Epoch 1/25
[[0.126318723 0.220419154 -0.0361928828 ... 0.118207403 -0.237727776 0.0547377095]
 [0.126318723 0.220419154 -0.0361928828 ... 0.118207403 -0.237727776 0.0547377095]
 [0.126318723 0.220419154 -0.0361928828 ... 0.118207403 -0.237727776 0.0547377095]
 ...
 [0.126318723 0.220419154 -0.0361928828 ... 0.118207403 -0.237727776 0.0547377095]
 [0.126318723 0.220419154 -0.0361928754 ... 0.118207403 -0.237727761 0.0547377095]
 [0.126318723 0.220419154 -0.0361928754 ... 0.118207403 -0.237727761 0.0547377095]]
1/8 [==>...........................] - ETA: 1s - loss: 2401526.5000[[0.103109382 0.195004448 -0.0372195616 ... 0.0995812565 -0.237570569 0.0376677513]
 [0.103109382 0.195004448 -0.0372195616 ... 0.0995812565 -0.237570569 0.0376677513]
 [0.103109382 0.195004448 -0.0372195616 ... 0.0995812565 -0.237570569 0.0376677513]
 ...
 [0.103109382 0.195004448 -0.0372195616 ... 0.0995812565 -0.237570569 0.0376677513]
 [0.103109382 0.195004478 -0.0372195616 ... 0.0995812491 -0.237570569 0.03766

[[-0.119637936 -0.120924287 0.0324541442 ... -0.0770445317 -0.229111567 -0.0320211723]
 [-0.119637936 -0.120924287 0.0324541442 ... -0.0770445317 -0.229111567 -0.0320211723]
 [-0.119637936 -0.120924287 0.0324541442 ... -0.0770445317 -0.229111567 -0.0320211723]
 ...
 [-0.119637936 -0.120924287 0.0324541442 ... -0.0770445317 -0.229111567 -0.0320211723]
 [-0.119637944 -0.120924287 0.0324541442 ... -0.0770445317 -0.229111567 -0.0320211761]
 [-0.119637944 -0.120924287 0.0324541442 ... -0.0770445317 -0.229111567 -0.0320211761]]
[[-0.122816108 -0.124187283 0.0313759744 ... -0.079645291 -0.228405654 -0.0350422338]
 [-0.122816108 -0.124187283 0.0313759744 ... -0.079645291 -0.228405654 -0.0350422338]
 [-0.122816108 -0.124187283 0.0313759744 ... -0.079645291 -0.228405654 -0.0350422338]
 ...
 [-0.122816108 -0.124187283 0.0313759744 ... -0.079645291 -0.228405654 -0.0350422338]
 [-0.122816108 -0.124187283 0.0313759744 ... -0.079645291 -0.228405654 -0.0350422338]
 [-0.122816108 -0.124187283 0.0313759

1/8 [==>...........................] - ETA: 0s - loss: 562.6062[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 

1/8 [==>...........................] - ETA: 0s - loss: 567.1083[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 

1/8 [==>...........................] - ETA: 0s - loss: 388.1480[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 

1/8 [==>...........................] - ETA: 0s - loss: 528.9121[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 

In [17]:
def my_evaluate():
    x_tensor = tf.convert_to_tensor(test_data, dtype=tf.float32)
    output = model(x_tensor)
    output = tf.reshape(output, [-1]).numpy()
    labels = test_labels[:,1]
    
    xx = output - labels
    
    ans = 0
    for n in xx:
        ans += n*n
    
    print(ans/len(xx))

my_evaluate()

134.79047586934257


In [ ]:
100 => 42.05 45.77

10 => 360, 575.477

15 -> 82.86, 170.013


In [ ]:
def grad(input_tensor, output_tensor):
    return tf.keras.layers.Lambda( lambda z: tf.keras.backend.gradients( z[ 0 ], z[ 1 ] ), output_shape = [1] )( [ output_tensor, input_tensor ] )

def custom_loss_wrapper(model, input_tensor):

    def custom_loss(y_true, y_pred):
        mse_loss = tf.keras.losses.mean_squared_error(y_true, y_pred)
        
        with tf.GradientTape() as tape:
            print(input_tensor)
            output = model(input_tensor)
            print(output)
        
#         print(tape.gradient(output, input_tensor))
            
#         derivative_loss = tf.keras.losses.mean_squared_error(input_tensor, -grad(input_tensor, grad(input_tensor, output_tensor))[0])
        return mse_loss

    return custom_loss

def my_loss_fn(y_true, y_pred):
    squared_difference = tf.square(y_true - y_pred)
    return tf.reduce_mean(squared_difference, axis=-1)

In [ ]:
## Creating own model

model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(7,)))
model.add(tf.keras.layers.Dense(12, activation='relu'))
model.add(tf.keras.layers.Dense(12, activation='relu'))
model.add(tf.keras.layers.Dense(1))


# model.summary() 
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss=custom_loss_wrapper(model, model.input))
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss='mean_squared_error')

model.fit(train_data, train_labels, epochs=10, batch_size=32, validation_split = 0.2, verbose = 0)

# print("Hi", model.input)

# input_m = model.input
# with tf.GradientTape() as tape:
# #     tape.watch(input_m)
#     output = model(input_m)
#     print(output)


In [ ]:
model.evaluate(test_data, test_labels)

In [ ]:
import numpy as np 

model.variables
# inp = tf.Variable(np.random.normal(size=(1, 7)), dtype=tf.float32)

# inp = tf.Variable(model.input)
# with tf.GradientTape() as tape:
#     preds = model(model.input)

# grads = tape.gradient(preds, model.input)
# print(grads)